In [44]:
import ast
import os

import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from collections import Counter
import statsmodels
import datetime
from sklearn.model_selection import train_test_split

df = pd.read_csv("metadata/ted_main.csv")
df['film_date'] = df['film_date'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
df['published_date'] = df['published_date'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
df['film_year'] = df['film_date'].apply(lambda it: it.year)
df['published_year'] = df['published_date'].apply(lambda it: it.year)

# Filter dataset
print(f"Total talks between 2010-2016: {len(df.loc[(df['film_year'] >= 2010) & (df['film_year'] <= 2016), :])}")

# We will use videos between 2010-2016
df = df.loc[(df['film_year'] >= 2010) & (df['film_year'] <= 2016), :]
# Num_speakers == 1
df = df.loc[df['num_speaker'] == 1, :]
print(len(df))
df

Total talks between 2010-2016: 1796
1758


,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,film_year,published_year
614,1137,Sharing powerful stories from his anti-obesity...,1313,TED2010,2010-02-20 02:00:00,49,Jamie Oliver,Jamie Oliver: Teach every child about food,1,2010-02-11 17:36:00,"[{'id': 23, 'name': 'Jaw-dropping', 'count': 1...","[{'id': 10, 'hero': 'https://pe.tedcdn.com/ima...","Chef, activist","['business', 'education', 'food', 'global issu...",Teach every child about food,https://www.ted.com/talks/jamie_oliver\n,7638978,2010,2010
615,209,"In a demo that drew gasps at TED2010, Blaise A...",465,TED2010,2010-02-11 02:00:00,28,Blaise Agüera y Arcas,Blaise Agüera y Arcas: Augmented-reality maps,1,2010-02-13 11:54:00,"[{'id': 23, 'name': 'Jaw-dropping', 'count': 8...","[{'id': 129, 'hero': 'https://pe.tedcdn.com/im...",Software architect,"['cities', 'design', 'map', 'technology', 'vir...",Augmented-reality maps,https://www.ted.com/talks/blaise_aguera\n,1718568,2010,2010
616,250,The leader of Britain's Conservative Party say...,839,TED2010,2010-02-10 02:00:00,29,David Cameron,David Cameron: The next age of government,1,2010-02-15 16:23:00,"[{'id': 8, 'name': 'Informative', 'count': 233...","[{'id': 604, 'hero': 'https://pe.tedcdn.com/im...",Politician,"['business', 'global issues', 'politics']",The next age of government,https://www.ted.com/talks/david_cameron\n,656762,2010,2010
618,948,"At TED2010, Bill Gates unveils his vision for ...",1669,TED2010,2010-02-12 02:00:00,38,Bill Gates,Bill Gates: Innovating to zero!,1,2010-02-18 03:00:00,"[{'id': 21, 'name': 'Unconvincing', 'count': 1...","[{'id': 51, 'hero': 'https://pe.tedcdn.com/ima...",Philanthropist,"['TED Brain Trust', 'business', 'energy', 'glo...",Innovating to zero!,https://www.ted.com/talks/bill_gates\n,4329332,2010,2010
620,277,The land of the free has become a legal minefi...,1101,TED2010,2010-02-13 02:00:00,24,Philip K. Howard,Philip K. Howard: Four ways to fix a broken le...,1,2010-02-21 11:15:00,"[{'id': 23, 'name': 'Jaw-dropping', 'count': 1...","[{'id': 187, 'hero': 'https://pe.tedcdn.com/im...",Legal activist,"['business', 'design', 'health care', 'law']",Four ways to fix a broken legal system,https://www.ted.com/talks/philip_howard\n,610454,2010,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525,18,Could it be wrong to help children in need by ...,791,TEDxSydney,2016-05-24 03:00:00,5,Tara Winkler,Tara Winkler: Why we need to end the era of or...,1,2017-08-29 23:00:46,"[{'id': 10, 'name': 'Inspiring', 'count': 119}...","[{'id': 1596, 'hero': 'https://pe.tedcdn.com/i...","Child protection leader, activist, author","['TEDx', 'activism', 'children', 'family', 'po...",Why we need to end the era of orphanages,https://www.ted.com/talks/tara_winkler_why_we_...,656113,2016,2017
2528,3,Digital archaeologist Chance Coughenour is usi...,717,TEDxHamburg,2016-06-08 03:00:00,5,Chance Coughenour,Chance Coughenour: How your pictures can help ...,1,2017-08-31 23:00:31,"[{'id': 9, 'name': 'Ingenious', 'count': 16}, ...","[{'id': 2673, 'hero': 'https://pe.tedcdn.com/i...",Digital archaeologist,"['TEDx', 'ancient world', 'archaeology', 'cons...",How your pictures can help reclaim lost history,https://www.ted.com/talks/chance_coughenour_ho...,539207,2016,2017
2529,45,"We all have origin stories and identity myths,...",1156,TEDxExeter,2015-04-24 03:00:00,1,Chetan Bhatt,Chetan Bhatt: Dare to refuse the origin myths ...,1,2017-09-01 17:29:03,"[{'id': 9, 'name': 'Ingenious', 'count': 24}, ...","[{'id': 2811, 'hero': 'https://pe.tedcdn.com/i...","Sociologist, human rights activist","['TEDx', 'community', 'humanity', 'identity', ...",Dare to refuse the origin myths that claim who...,https://www.ted.com/talks/chetan_bhatt_dare_to...,857850,2015,2017
2531,18,Terrorists and extremists aren't all naturally...,698,TEDxGhent,2016-06-18 03:00:00,1,Erin Marie Saltman,Erin Marie

In [45]:
# Generate target set
log_views = np.log(df['views'])
df['log_views_norm'] = (log_views - log_views.mean()) / log_views.std()


def map_to_label(value):
    if value < -1.5:
        return "very_low"
    elif value < -0.5:
        return "low"
    elif value < 0.5:
        return "medium"
    elif value < 1.5:
        return "high"
    else:
        return "very_high"


df['log_views_norm_cat'] = df['log_views_norm'].apply(map_to_label)

In [66]:
merged_metadata_df = pd.read_csv("metadata/merged_metadata_popularity.csv")
merged_metadata_df

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,tags,title,url,views,transcript,filename,film_year,published_year,log_views_norm,log_views_norm_cat
0,1137,Sharing powerful stories from his anti-obesity...,1313,TED2010,2010-02-20 02:00:00,49,Jamie Oliver,Jamie Oliver: Teach every child about food,1,2010-02-11 17:36:00,...,"['business', 'education', 'food', 'global issu...",Teach every child about food,https://www.ted.com/talks/jamie_oliver,7638978,"Sadly, in the next 18 minutes when I do our ch...",2010-jamie-oliver-007-fallback-93948ab6f9d8306...,2010,2010,2.595681,high
1,209,"In a demo that drew gasps at TED2010, Blaise A...",465,TED2010,2010-02-11 02:00:00,28,Blaise Agüera y Arcas,Blaise Agüera y Arcas: Augmented-reality maps,1,2010-02-13 11:54:00,...,"['cities', 'design', 'map', 'technology', 'vir...",Augmented-reality maps,https://www.ted.com/talks/blaise_aguera,1718568,"About a year and a half ago, Stephen Lawler, w...",2010-blaise-aguera-y-arcas-016-fallback-a39ea7...,2010,2010,0.369330,normal
2,250,The leader of Britain's Conservative Party say...,839,TED2010,2010-02-10 02:00:00,29,David Cameron,David Cameron: The next age of government,1,2010-02-15 16:23:00,...,"['business', 'global issues', 'politics']",The next age of government,https://www.ted.com/talks/david_cameron,656762,"Someone once said that politics is, of course,...",2010-david-cameron-009-fallback-e23a18509bb2a0...,2010,2010,-1.066266,low
3,948,"At TED2010, Bill Gates unveils his vision for ...",1669,TED2010,2010-02-12 02:00:00,38,Bill Gates,Bill Gates: Innovating to zero!,1,2010-02-18 03:00:00,...,"['TED Brain Trust', 'business', 'energy', 'glo...",Innovating to zero!,https://www.ted.com/talks/bill_gates,4329332,I'm going to talk today about energy and clima...,2010-bill-gates-006-fallback-69469ceb41a7e17ed...,2010,2010,1.748209,high
4,277,The land of the free has become a legal minefi...,1101,TED2010,2010-02-13 02:00:00,24,Philip K. Howard,Philip K. Howard: Four ways to fix a broken le...,1,2010-02-21 11:15:00,...,"['business', 'design', 'health care', 'law']",Four ways to fix a broken legal system,https://www.ted.com/talks/philip_howard,610454,I've always been interested in the relationshi...,2010-philip-k-howard-005-fallback-a9ddfb85ca66...,2010,2010,-1.175390,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,18,Could it be wrong to help children in need by ...,791,TEDxSydney,2016-05-24 03:00:00,5,Tara Winkler,Tara Winkler: Why we need to end the era of or...,1,2017-08-29 23:00:46,...,"['TEDx', 'activism', 'children', 'family', 'po...",Why we need to end the era of orphanages,https://www.ted.com/talks/tara_winkler_why_we_...,656113,These are some photos of me volunteering in a ...,2016x-tara-winkler-008-fallback-88253cf89ec47f...,2016,2017,-1.067742,low
1698,3,Digital archaeologist Chance Coughenour is usi...,717,TEDxHamburg,2016-06-08 03:00:00,5,Chance Coughenour,Chance Coughenour: How your pictures can help ...,1,2017-08-31 23:00:31,...,"['TEDx', 'ancient world', 'archaeology', 'cons...",How your pictures can help reclaim lost history,https://www.ted.com/talks/chance_coughenour_ho...,539207,Why do people deliberately destroy cultural he...,2016x-chance-coughenour-006-fallback-dbb5bbc79...,2016,2017,-1.360604,low
1699,45,"We all have origin stories and identity myths,...",1156,TEDxExeter,2015-04-24 03:00:00,1,Chetan Bhatt,Chetan Bhatt: Dare to refuse the origin myths ...,1,2017-09-01 17:29:03,...,"['TEDx', 'community', 'humanity', 'identity', ...",Dare to refuse the origin myths that claim who...,https://www.ted.com/talks/chetan_bhatt_dare_to...,857850,"I'm Chetan Bhatt and when I give my name, I'm ...",2015x-chetan-bhatt-004-fallback-e542bdf061f08d...,2015,2017,-0.667629,low
1700,18,Terrorists and extremists aren't all naturally...,698,TEDxGhent,2016-06-18 03:00:00,1,Erin Marie Saltman,Erin Marie Saltman: How young people join viol...,1,2017-09-05 

In [67]:
# Concatenate with features
import json

def get_related_result(row):
    for entry in os.listdir('modeling_api_results'):
        if not entry.endswith(".wav_summary.json"):
            continue
        
        if entry.split(".wav_summary.json")[0] == row['filename'].split(".wav")[0]:
            with open(os.path.join('modeling_api_results', entry)) as f:
                return json.load(f)
            
    # print(f"File {row['url']} not found in modeling_api_results")
    return None


features_cols = {
    "emotion_angry": [],
    "emotion_happy": [],
    "emotion_neutral": [],
    "emotion_sad": [],
    "strength_weak": [],
    "strength_neutral": [],
    "strength_strong": [],
    "positivity_negative": [],
    "positivity_neutral": [],
    "positivity_positive": [],
}

for idx, row in merged_metadata_df.iterrows():
    try:
        # Modeling api result
        result = get_related_result(row)
    
        # Aggregate with Thodoris script
        # TODO
        for k, v in features_cols.items():
            task, cls = k.split("_")
            
            if result is None:
                v.append(pd.NA)
            else:
                total = np.sum(list(result[task].values()))
                value = result[task].get(cls, 0)
                v.append(value/total)
    except Exception:
        print(result, row['filename'])
        for k, v in features_cols.items():
            v.append(pd.NA)
            
for k, v in features_cols.items():
    merged_metadata_df[k] = v
    
features_metadata = merged_metadata_df.dropna()
features_metadata

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,emotion_angry,emotion_happy,emotion_neutral,emotion_sad,strength_weak,strength_neutral,strength_strong,positivity_negative,positivity_neutral,positivity_positive
0,1137,Sharing powerful stories from his anti-obesity...,1313,TED2010,2010-02-20 02:00:00,49,Jamie Oliver,Jamie Oliver: Teach every child about food,1,2010-02-11 17:36:00,...,0.373418,0.335443,0.278481,0.012658,0.006329,0.341772,0.651899,0.221519,0.746835,0.031646
1,209,"In a demo that drew gasps at TED2010, Blaise A...",465,TED2010,2010-02-11 02:00:00,28,Blaise Agüera y Arcas,Blaise Agüera y Arcas: Augmented-reality maps,1,2010-02-13 11:54:00,...,0.0,0.233333,0.766667,0.0,0.033333,0.883333,0.083333,0.0,0.933333,0.066667
2,250,The leader of Britain's Conservative Party say...,839,TED2010,2010-02-10 02:00:00,29,David Cameron,David Cameron: The next age of government,1,2010-02-15 16:23:00,...,0.008696,0.026087,0.965217,0.0,0.0,0.947826,0.052174,0.0,0.991304,0.008696
3,948,"At TED2010, Bill Gates unveils his vision for ...",1669,TED2010,2010-02-12 02:00:00,38,Bill Gates,Bill Gates: Innovating to zero!,1,2010-02-18 03:00:00,...,0.09542,0.167939,0.736641,0.0,0.003817,0.469466,0.526718,0.068702,0.923664,0.007634
4,277,The land of the free has become a legal minefi...,1101,TED2010,2010-02-13 02:00:00,24,Philip K. Howard,Philip K. Howard: Four ways to fix a broken le...,1,2010-02-21 11:15:00,...,0.0125,0.03125,0.925,0.03125,0.0,0.875,0.125,0.025,0.975,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,94,"Sheikha Al Mayassa, a patron of artists, story...",653,TEDWomen 2010,2010-12-02 02:00:00,28,Sheikha Al Mayassa,"Sheikha Al Mayassa: Globalizing the local, loc...",1,2012-02-08 18:28:15,...,0.0,0.170213,0.819149,0.010638,0.0,0.287234,0.712766,0.042553,0.925532,0.031915
446,78,"In a spellbinding talk, paleontologist Jack Ho...",1103,TEDxVancouver,2011-11-11 02:00:00,28,Jack Horner,Jack Horner: Where are the baby dinosaurs?,1,2012-02-09 17:59:58,...,0.0,0.046512,0.953488,0.0,0.100775,0.852713,0.046512,0.0,0.992248,0.007752
448,267,We have all changed someone's life -- usually ...,374,TEDxToronto 2010,2010-09-23 03:00:00,38,Drew Dudley,Drew Dudley: Everyday leadership,1,2012-02-11 17:03:50,...,0.338462,0.107692,0.553846,0.0,0.0,0.307692,0.692308,0.323077,0.661538,0.015385
451,44,"They're the second-largest fish in the world, ...",1006,TEDxDublin,2010-03-29 03:00:00,25,Simon Berrow,Simon Berrow: How do you save a shark you know...,1,2012-02-16 17:52:00,...,0.0,0.013072,0.986928,0.0,0.0,0.993464,0.006536,0.0,0.993464,0.006536


In [120]:
# Find the most appeared tags
total_tags = [tag for row in features_metadata.tags for tag in ast.literal_eval(row)]
tags_counter = Counter(total_tags).most_common()
most_common_tags = [t[0] for t in tags_counter[:10] if t[0] != 'TEDx']

active_tags = most_common_tags
print(f"Active tags: {active_tags}")

dominant_group = []
for idx, row in features_metadata.iterrows():
    entry = pd.NA
    for common_tag in reversed(active_tags):
        for tag in ast.literal_eval(row['tags']):
            if tag == common_tag and pd.isna(entry):
                entry = tag
    dominant_group.append(entry)

features_metadata['dominant_tag'] = dominant_group

Active tags: ['culture', 'technology', 'global issues', 'science', 'business', 'design', 'entertainment', 'education', 'health']


/var/folders/sz/fx1sx7950pj622kmk32jnvcw0000gp/T/ipykernel_44456/1173650416.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [125]:
import plotly.express as px

topic_metadata = features_metadata.loc[:, :]


active_feats = ['emotion_happy', 'strength_strong', 'positivity_positive']
sum_of_feats = np.sum([topic_metadata[k] for k in active_feats], axis=0)

X = np.asarray(sum_of_feats).astype(np.float64)
Y = np.asarray(topic_metadata['log_views_norm']).astype(np.float64)

idx = np.where(X > 0)[0]
X = X[idx]
Y = Y[idx]

X = np.log(X)
fig = px.scatter(x=X, y=Y, trendline="ols")
fig.data[1].line.color = 'red'
fig.show()

In [123]:
topic_metadata[['log_views_norm'] + list(features_cols.keys()) ].corr()

,log_views_norm,emotion_angry,emotion_happy,emotion_neutral,emotion_sad,strength_weak,strength_neutral,strength_strong,positivity_negative,positivity_neutral,positivity_positive
log_views_norm,1.000000,0.012061,-0.063103,0.068374,-0.112371,-0.057128,-0.178792,0.198058,-0.055037,0.278985,-0.265382
emotion_angry,0.012061,1.000000,-0.021613,-0.184794,-0.162429,-0.150231,-0.485516,0.536175,0.785635,-0.233455,-0.146398
emotion_happy,-0.063103,-0.021613,1.000000,-0.975621,-0.095287,0.076713,-0.481715,0.455494,-0.130810,-0.696705,0.796380
emotion_neutral,0.068374,-0.184794,-0.975621,1.000000,0.052353,-0.057402,0.591515,-0.571782,-0.046324,0.743449,-0.757189
emotion_sad,-0.112371,-0.162429,-0.095287,0.052353,1.000000,0.187450,-0.123429,0.059867,0.048021,-0.054988,0.033787
strength_weak,-0.057128,-0.150231,0.076713,-0.057402,0.187450,1.000000,-0.168038,-0.170814,-0.128386,-0.136915,0.207660
strength_neutral,-0.178792,-0.485516,-0.481715,0.591515,-0.123429,-0.168038,1.000000,-0.942590,-0.400657,0.396678,-0.216710
strength_strong,0.198058,0.536175,0.455494,-0.571782,0.059867,-0.170814,-0.942590,1.000000,0.443957,-0.350103,0.146256
positivity_negative,-0.055037,0.785635,-0.130810,-0.046324,0.048021,-0.128386,-0.400657,0.443957,1.000000,-0.334411,-0.147240
positivity_neutral,0.278985,-0.233455,-0.696705,0.743449,-0.054988,-0.136915,0.396678,-0.350103,-0.334411,1.000000,-0.882916


In [121]:
features_metadata

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,emotion_happy,emotion_neutral,emotion_sad,strength_weak,strength_neutral,strength_strong,positivity_negative,positivity_neutral,positivity_positive,dominant_tag
0,1137,Sharing powerful stories from his anti-obesity...,1313,TED2010,2010-02-20 02:00:00,49,Jamie Oliver,Jamie Oliver: Teach every child about food,1,2010-02-11 17:36:00,...,0.335443,0.278481,0.012658,0.006329,0.341772,0.651899,0.221519,0.746835,0.031646,health
1,209,"In a demo that drew gasps at TED2010, Blaise A...",465,TED2010,2010-02-11 02:00:00,28,Blaise Agüera y Arcas,Blaise Agüera y Arcas: Augmented-reality maps,1,2010-02-13 11:54:00,...,0.233333,0.766667,0.0,0.033333,0.883333,0.083333,0.0,0.933333,0.066667,design
2,250,The leader of Britain's Conservative Party say...,839,TED2010,2010-02-10 02:00:00,29,David Cameron,David Cameron: The next age of government,1,2010-02-15 16:23:00,...,0.026087,0.965217,0.0,0.0,0.947826,0.052174,0.0,0.991304,0.008696,business
3,948,"At TED2010, Bill Gates unveils his vision for ...",1669,TED2010,2010-02-12 02:00:00,38,Bill Gates,Bill Gates: Innovating to zero!,1,2010-02-18 03:00:00,...,0.167939,0.736641,0.0,0.003817,0.469466,0.526718,0.068702,0.923664,0.007634,business
4,277,The land of the free has become a legal minefi...,1101,TED2010,2010-02-13 02:00:00,24,Philip K. Howard,Philip K. Howard: Four ways to fix a broken le...,1,2010-02-21 11:15:00,...,0.03125,0.925,0.03125,0.0,0.875,0.125,0.025,0.975,0.0,design
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,94,"Sheikha Al Mayassa, a patron of artists, story...",653,TEDWomen 2010,2010-12-02 02:00:00,28,Sheikha Al Mayassa,"Sheikha Al Mayassa: Globalizing the local, loc...",1,2012-02-08 18:28:15,...,0.170213,0.819149,0.010638,0.0,0.287234,0.712766,0.042553,0.925532,0.031915,culture
446,78,"In a spellbinding talk, paleontologist Jack Ho...",1103,TEDxVancouver,2011-11-11 02:00:00,28,Jack Horner,Jack Horner: Where are the baby dinosaurs?,1,2012-02-09 17:59:58,...,0.046512,0.953488,0.0,0.100775,0.852713,0.046512,0.0,0.992248,0.007752,science
448,267,We have all changed someone's life -- usually ...,374,TEDxToronto 2010,2010-09-23 03:00:00,38,Drew Dudley,Drew Dudley: Everyday leadership,1,2012-02-11 17:03:50,...,0.107692,0.553846,0.0,0.0,0.307692,0.692308,0.323077,0.661538,0.015385,<NA>
451,44,"They're the second-largest fish in the world, ...",1006,TEDxDublin,2010-03-29 03:00:00,25,Simon Berrow,Simon Berrow: How do you save a shark you know...,1,2012-02-16 17:52:00,...,0.013072,0.986928,0.0,0.0,0.993464,0.006536,0.0,0.993464,0.006536,science
